In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from time import time 

In [ ]:
df = pd.read_csv('../input/data-analyst-jobs/DataAnalyst.csv')
df.head()

In [ ]:
df = df[['Job Title', 'Job Description']]
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser'])

In [ ]:
def cleaning(doc):
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Job Description'])

In [ ]:
print(df['Job Description'])

In [ ]:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
import gensim 
from gensim.models import Word2Vec

In [ ]:
sent = [row.split() for row in df_clean['clean']]

In [ ]:
print(sent[:10])

In [ ]:
t = time()

model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
model.wv.most_similar(positive=["data"])

In [ ]:
model.wv.index2entity[:50] #retrieving the 50 most common words in the corpus

In [ ]:
model.wv.similarity("sql", 'python') #how similar are thise languages?

In [ ]:
model.wv.doesnt_match(['report', 'analysis', 'salary']) #which word doesn't fit?